# Quadratic Assignment Problem

Notebook where we have taken the code from: https://github.com/recruit-communications/pyqubo/blob/master/notebooks/TSP.ipynb and modified it to solve a QAP

In [1]:
%matplotlib inline
from datetime import datetime

import neal
import numpy as np
import pandas as pd
from itertools import permutations
from dwave.system import EmbeddingComposite, DWaveSampler
from pyqubo import Array, Placeholder, Constraint

## Facilities assignment to locations (given their distances and flows between them) - QAP

### Number of locations/facilities = 3

[[0 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### Number of cities/facilities = 4

In [2]:
# locations (locs) and facilities should be same-sized lists
locs = list('abcd')
facilities = ['hq', 'rnd', 'depo', 'factory']

In [3]:
# size of the problem to consider - useful to consider smaller problem to that solves quickly (cannot be larger than len(locs)!)
m = 4  

In [4]:
dist_bw_cities = np.array([[0, 10, 40, 30], [10, 0, 20, 50], [40, 20, 0, 20], [30, 50, 20, 0]])
dbc = pd.DataFrame(dist_bw_cities[:m, :m], index=locs[:m], columns=locs[:m])
print("Distance between cities:")
print(dbc)

Distance between cities:
    a   b   c   d
a   0  10  40  30
b  10   0  20  50
c  40  20   0  20
d  30  50  20   0


In [5]:
flows_bw_facilities = np.array([[0, 30, 5, 10], [30, 0, 5, 20], [5, 5, 0, 50], [10, 20, 50, 0]])
fbf = pd.DataFrame(flows_bw_facilities[:m, :m], index=facilities[:m], columns=facilities[:m])
print("Flows between facilities:")
print(fbf)

Flows between facilities:
         hq  rnd  depo  factory
hq        0   30     5       10
rnd      30    0     5       20
depo      5    5     0       50
factory  10   20    50        0


## Instantiating variables (Common code for simulated and quantum annealing)

### Unknown variable i.e. $x$

The unknown variable we're solving for $x$ represents the binary vector with  bit $(i, j)$ representing assignment of facility $i$ to location $j$ 

In [6]:
n_city = m
x = Array.create('x', (n_city, n_city), 'BINARY')

### Objective Function

Objective function to be minimised - $flow*distance$ for chosen $x$
$$
H_{1} = min\sum_{i=1}^{n}\sum_{j=1}^{n}\sum_{k=1}^{n}\sum_{l=1}^{n}(f_{ij}.d_{kl})x_{i,k}x_{j,l}
$$
with $x_{i,k}$ being a binary variable, which is $1$ if facility $i$ is assigned to location $k$ and $0$ otherwise. The same applies for the binary variable $x_{j,l}$

In [7]:
flow_times_dist = 0.0
for i in range(n_city):
    for j in range(n_city):
        for k in range(n_city):
            for l in range(n_city):
                flow_times_dist += flows_bw_facilities[i][j] * x[i][k] * dist_bw_cities[k][l] * x[j][l]  
                                # the last is x_jl because we've X' there

In [8]:
# print(flow_times_dist)

### Constraints 

$$
H_{2} = P\sum_{i=1}^{n} (1 - \sum_{k=1}^{n}x_{i,k})^2 + P\sum_{l=1}^{n} (1 - \sum_{j=1}^{n}x_{j,l})^2
$$

Constraint not to have $x[i,j]$ be different from 1 in more than one column in each row.

In [9]:
row_const = 0.0
for i in range(n_city):
    # If you wrap the hamiltonian by Const(...), this part is recognized as constraint
    row_const += Constraint((sum(x[i, j] for j in range(n_city)) - 1) ** 2, label="row{}".format(i))

In [10]:
# print(row_const)

Constraint not to have $x[i,j]$ be different from 1 in more than one row in each column.

In [11]:
column_const = 0.0
for j in range(n_city):
    column_const += Constraint((sum(x[i, j] for i in range(n_city)) - 1) ** 2, label="column{}".format(j))

In [12]:
# print(column_const)

### Penalty value

The penalty value $P$ of the constraint Hamiltonian $H_{2}$ is set to $P > d_{max}*f_{max}$

In [13]:
# Construct hamiltonian
P = Placeholder("P")

In [14]:
max_bound = np.max(dist_bw_cities) * np.max(flows_bw_facilities) + 1
print(f'max_bound: {max_bound}')

max_bound: 2501


In [15]:
feed_dict = {'P': max_bound}

### Generate QUBO

The overall Hamiltonian is composed of $H = H_{1} + H_{2}$

In [16]:
H = flow_times_dist + P * (row_const + column_const)

In [17]:
# print(H)

In [18]:
# Compile model
model = H.compile()

## Simulated Annealing

In [32]:
bqm = model.to_bqm(feed_dict=feed_dict)

In [33]:
t_start = datetime.now()
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=100)

# Decode solution from sampleset
decoded_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
print(f'total decoded_samples: {len(decoded_samples)}\n')

# Optimal solution with minimum energy state
best_sample = min(decoded_samples, key=lambda x: x.energy)

# Checking the ratio of constraints that were not satisfied for the optimal solution
num_broken = len(best_sample.constraints(only_broken=True))
tot_constraints = len(best_sample.constraints())
print("number of broken constraints = {} out of {}\n".format(num_broken, tot_constraints))


if num_broken == 0:
    print(best_sample)
else:
    print(best_sample.constraints())
    
print(f'\ntime taken (using SimulatedAnnealing): {datetime.now() - t_start}')


total decoded_samples: 100

number of broken constraints = 0 out of 8

DecodedSolution({x[3][1]:1, x[0][0]:0, x[3][0]:0, x[0][1]:0, x[3][3]:0, x[0][2]:0, x[3][2]:0, x[0][3]:1, x[1][0]:0, x[1][1]:0, x[1][2]:1, x[1][3]:0, x[2][0]:1, x[2][1]:0, x[2][2]:0, x[2][3]:0}, energy=4700.000000)

time taken (using SimulatedAnnealing): 0:00:00.181574


In [21]:
# Optimal solution as a 2D matrix
opt = np.zeros((n_city, n_city))
for i in range(n_city):
    for j in range(n_city):
        if best_sample.array('x', (i, j)) == 1:
            opt[i][j] = 1

print(opt)

[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]


In [22]:
# Cost (flow*distance) for travel between each facility 
print(flows_bw_facilities * (opt @ dist_bw_cities @ opt.T))

[[  0. 600. 150. 500.]
 [600.   0. 200. 400.]
 [150. 200.   0. 500.]
 [500. 400. 500.   0.]]


In [23]:
optimal_solution = 0.5 * best_sample.energy  # because we need to consider the flow*dist only once (upper or lower half of matrix)
print(f'optimal solution (minimum of flow*dist possible): {optimal_solution}')

optimal solution (minimum of flow*dist possible): 2350.0


## Quantum Annealing using D-wave sampler

In [24]:
ENDPOINT = 'https://cloud.dwavesys.com/sapi/'
SOLVER = 'Advantage_system6.1'
TOKEN = '***' # Enter token 

In [25]:
num_reads = 100

In [26]:
dw_solver = DWaveSampler(solver=SOLVER, token=TOKEN, endpoint=ENDPOINT)
dwave = EmbeddingComposite(dw_solver)

Q, offset = model.to_qubo(feed_dict=feed_dict)

sampleset = dwave.sample_qubo(Q, num_reads=100)

# Decode solution from sampleset
decoded_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
print(f'total decoded_samples: {len(decoded_samples)}\n')

# Optimal solution with minimum energy state
best_sample = min(decoded_samples, key=lambda x: x.energy)

# Checking the ratio of constraints that were not satisfied for the optimal solution
num_broken = len(best_sample.constraints(only_broken=True))
tot_constraints = len(best_sample.constraints())
print("number of broken constraints = {} out of {}\n".format(num_broken, tot_constraints))


if num_broken == 0:
    print(best_sample)
else:
    print(best_sample.constraints())
    
print(f'\ntime taken (using QuantumAnnealing): ')
print(sampleset.info)

total decoded_samples: 76

number of broken constraints = 0 out of 8

DecodedSolution({x[0][0]:0, x[3][1]:1, x[0][1]:0, x[3][0]:0, x[0][2]:0, x[3][3]:0, x[0][3]:1, x[3][2]:0, x[1][0]:0, x[1][1]:0, x[1][2]:1, x[1][3]:0, x[2][0]:1, x[2][1]:0, x[2][2]:0, x[2][3]:0}, energy=4700.000000)

time taken (using QuantumAnnealing): 
{'timing': {'qpu_sampling_time': 9502.0, 'qpu_anneal_time_per_sample': 20.0, 'qpu_readout_time_per_sample': 54.48, 'qpu_access_time': 25427.57, 'qpu_access_overhead_time': 645.43, 'qpu_programming_time': 15925.57, 'qpu_delay_time_per_sample': 20.54, 'post_processing_overhead_time': 1766.0, 'total_post_processing_time': 1766.0}, 'problem_id': '96dd1308-62b4-4168-8412-c61d456bf76f'}


In [27]:
# Optimal solution as a 2D matrix
opt = np.zeros((n_city, n_city))
for i in range(n_city):
    for j in range(n_city):
        if best_sample.array('x', (i, j)) == 1:
            opt[i][j] = 1

print(opt)

[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]


In [28]:
# Cost (flow*distance) for travel between each facility 
print(flows_bw_facilities * (opt @ dist_bw_cities @ opt.T))

[[  0. 600. 150. 500.]
 [600.   0. 200. 400.]
 [150. 200.   0. 500.]
 [500. 400. 500.   0.]]


In [29]:
optimal_solution = 0.5 * best_sample.energy  # because we need to consider the flow*dist only once (upper or lower half of matrix)
print(f'optimal solution (minimum of flow*dist possible): {optimal_solution}')

optimal solution (minimum of flow*dist possible): 2350.0


## Classical method

In [30]:
# Create permutation matrices and find the total cost i.e. flow*distance, associated with the matrix
permutation_mapping = {1: [1, 0, 0, 0], 2: [0, 1, 0, 0], 3: [0, 0, 1, 0], 4: [0, 0, 0, 1]}

cost_matrix_mapping = {}
for p in list(permutations(range(1, 5))):
    permutation_matrix = np.array([permutation_mapping[x] for x in p]).reshape(4, 4)
    cost = np.sum(flows_bw_facilities * (permutation_matrix @ dist_bw_cities @ permutation_matrix.T))/2
    cost_matrix_mapping[p] = cost

In [31]:
# Find the permutation matrix with the lowest cost 

opt_permutation = min(cost_matrix_mapping, key=cost_matrix_mapping.get)
opt = np.array([permutation_mapping[x] for x in opt_permutation]).reshape(4, 4)

# Optimal solution as a 2D matrix
print(opt)

[[0 0 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]]
